In [11]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text
import geopandas as gpd
from shapely import wkt
from shapely.geometry import LineString, shape, MultiPolygon
import shapely
import os
import openpyxl
import glob
import geopandas
import ast
import numpy as np

# Regl til vask


Dette Script retter de objekter som løber under højspændingsledningerne således at de passer på reglen: Højspænding skal krydse > 5 meter over andre objekter !

# Connection to Database, Create engine

In [18]:
from sqlalchemy.engine import create_engine

DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
HOST = 'exal6c01-scan.prod.sitad.dk' #enter the oracle db host url
PORT = 1521 # enter the oracle port number
SERVICE = 'GEOBANK.prod.sitad.dk' # enter the oracle db service name
ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH_WIN_AUTH, pool_size=50, max_overflow=0)



# Load De Objekter som krydser

In [19]:
headers = ['fejltype',
           'tekst',
           'fejl',
           'størrelse',
           'Objekter',
           'Sted',
           'Kommune']

# File from Desktrop, if the latest_file is being used it cant be asseced. 
fp_desktop = 'C:/Users/B177999/Desktop/Fejlsteder.xlsx'

Fejlsteder = pd.read_excel(fp_desktop, header=None, names=headers)

Fejlsteder = Fejlsteder.drop(0)

In [20]:
# you can use the ast.literal_eval function from the ast module in Python to convert a string representation of a list to an actual list object.
Fejlsteder['Objekter'] = Fejlsteder['Objekter'].apply(lambda x: ast.literal_eval(x))

# Get Hoejspaendings Objekter
Fejlsteder['Objekter_1'] = Fejlsteder['Objekter'].map(lambda x: x[1])

# Get the objekter running below Hoejspaending
Fejlsteder['Objekter_2'] = Fejlsteder['Objekter'].map(lambda x: x[0])

In [21]:
# collumns to string
Fejlsteder['Objekter_1'] = Fejlsteder['Objekter_1'].astype(str)
Fejlsteder['Objekter_2'] = Fejlsteder['Objekter_2'].astype(str)

# Objekter_1 Query

In [22]:
# Load all tabel names from the Database

query = """select col.owner as schema_name,
       col.table_name,
       column_id,
       column_name,
       data_type
from sys.all_tab_cols col
join sys.all_tables tab on col.owner = tab.owner
                        and col.table_name = tab.table_name
where col.data_type = 'SDO_GEOMETRY'
      and col.owner not in 
      ('ANONYMOUS','CTXSYS','DBSNMP','EXFSYS', 'LBACSYS', 
      'MDSYS', 'MGMT_VIEW','OLAPSYS','OWBSYS','ORDPLUGINS', 'ORDSYS',
      'SI_INFORMTN_SCHEMA','SYS','SYSMAN','SYSTEM', 'TSMSYS','WK_TEST',
      'WKPROXY','WMSYS','XDB','APEX_040000', 'APEX_PUBLIC_USER','DIP', 
      'FLOWS_30000','FLOWS_FILES','MDDATA', 'ORACLE_OCM', 'XS$NULL',
      'SPATIAL_CSW_ADMIN_USR', 'SPATIAL_WFS_ADMIN_USR', 'PUBLIC',
      'OUTLN', 'WKSYS', 'APEX_040200')
order by col.owner,
         col.table_name,
         column_id"""

df_oracle = pd.read_sql_query(sql=text(query), con=engine.connect())

In [23]:
GEODANMARK_60_B = []
def table_name(schema_name, table_name):
    if schema_name == 'GEODANMARK_60_B':
        GEODANMARK_60_B.append(table_name)
df_oracle.apply(lambda x: table_name(x['schema_name'],x['table_name']),axis=1)
print(len(GEODANMARK_60_B))

70


In [24]:
# Disse Id'er bruges til vores Query
for element in Fejlsteder['Objekter_1'].unique():
    to_query = element + ','
    #print(to_query)

## Load Objekter_1

In [25]:
Query_list = []
for tabel in GEODANMARK_60_B:
    Query = """SELECT '{}' AS TABEL, ID_LOKALID, SDO_UTIL.TO_WKTGEOMETRY(geometri) AS geometri FROM GEODANMARK_60_NOHIST.{}
WHERE
    ID_LOKALID IN (
    1102696460,
1003261107,
1003261116,
1098167019,
1098167021,
1114646543,
1098266252,
1009821390,
1098167020,
1113476582,
1212095582,
1003261153,
1097206831,
1003261162,
1114573318,
1003261135,
1003261115,
1101289890,
1013982201,
1054738211,
1108189127,
1098266249,
1214923335,
1003261158,
1041383051,
1003261144,
1003261145,
1003261130,
1038233525,
1114646978,
1054738216,
1114647057,
1214923336,
1054738223,
1050131708,
1039378293,
1098174477,
1219692361,
1003261191,
1003261199,
1003261125,
1061696350,
1055883089,
1114646977,
1214923339,
1003261184,
1003261126,
1061709583,
1056050094,
1114714155,
1003261146,
1062563529,
1061697066,
1062563535,
1099111546,
1093377286,
1003261217,
1003261210,
1003261209,
1061697065,
1055848036,
1091515884,
1055404656,
1055399466,
1213688968,
1091515883,
1059167074,
1044689181,
1114647206,
1108217077,
1114721649,
1056083376,
1100450582,
1102696463,
1114713668,
1114713669,
1114650130,
1044723197,
1094630127,
1044689175,
1040579137,
1099990346,
1114713665,
1107198881,
1113242867,
1083205314,
1061356191,
1062563533,
1114606454,
1059690829,
1019275574,
1105831196,
1057985288,
1060429515,
1007154741,
1114662265,
1113242863,
1061368761,
1114650012,
1095068820,
1098324755,
1098324699,
1037934783,
1037954280,
1037954565,
1114713671,
1114713670,
1062563537,
1007729347,
1108189800,
1105291884,
1114671422,
1014011732,
1013998049,
1032951881,
1033084345,
1033622337,
1033261725,
1114648857,
1114714185,
1006983406,
1052320831,
1029400122,
1107944551,
1007729405,
1114713664,
1007154749,
1007154748,
1114662264,
1017544677,
1105109737,
1091272001,
1098322869,
1098325473,
1033064557,
1036633144,
1098324718,
1046694075,
1006983341,
1219438369,
1098052581,
1098052578,
1017297754,
1019273645,
1046694074,
1046694072,
1008605861,
1008605859,
1008605061,
1008605860,
1102696447,
1008380868,
1008605871,
1019275556,
1107037826,
1098322888,
1098322826,
1034597692,
1034597646,
1033922892,
1033677274,
1033677269,
1033920153,
1098324685,
1008605870,
1008605867,
1107943024,
1008605866,
1008409312,
1008408968,
1008409122,
1037044870,
1098324782,
1098324591,
1090668243,
1008409362,
1008409363,
1068399162,
1008409361,
1008409313,
1021498863,
1025982081,
1025982067,
1025982084,
1054108205,
1060641347,
1065976634,
1024665061,
1024665060,
1024665689,
1207665358,
1060641355,
1066132790,
1060641353,
1021236217,
1026752201,
1030681030,
1114608944,
1065866524,
1107940188,
1107940277,
1026752202,
1026729667,
1065866798,
1022743589,
1108223692,
1065866154,
1060641354,
1114609039,
1083054363,
1025447426,
1024856222,
1113750464,
1107197950,
1113750461,
1114609029,
1107196733,
1107894230,
1091276778,
1026752205,
1108223693,
1113750462,
1114609025,
1010697788,
1010697793,
1107893547,
1107148664,
1023540474,
1113761503,
1078487656,
1031586717,
1031651983,
1031651982,
1031651984,
1031581783,
1219748785,
1113476608,
1207081340,
1003280839,
1003279947,
1003164046,
1040905815,
1002620233,
1040905769,
1205545285,
1040835534,
1040877649,
1107942910,
1208533821,
1009971263,
1009915650,
1009914798,
1219614202,
1009795950,
1009796014,
1114647219,
1107142564,
1009938489,
1009713266,
1009713604,
1009713238,
1040113602,
1040148544,
1003261170,
1114146133,
1207665325,
1102696466,
1099111548,
1213688971,
1055399468,
1055399469,
1099111552,
1216643169,
1058774480,
1058770694,
1216641920,
1216642002,
1099135245,
1216641893,
1099107907,
1216643170,
1216643171,
1211632025,
1216643159,
1216641825,
1216643163,
1216643162,
1058678041,
1216642078,
1058683854,
1058686265,
1009684457,
1009995619,
1009995604,
1202210777,
1102475585,
1114647340,
1041850051,
1219452582,
1003261103,
1107943842,
1041823725,
1100086862,
1211312791,
1041914107,
1094761786,
1096022113,
1041850053,
1096021635,
1212095580,
1003261192,
1061671729,
1107943021,
1093377287,
1057338525,
1057099728,
1057338706,
1114647843,
1114606602,
1015859432,
1055820780,
1213688969,
1107943908,
1055820851,
1091518544,
1108217193,
1055363634,
1055296633,
1055404650,
1055729261,
1213682703,
1055363467,
1114606606,
1211303615,
1094244676,
1093388986,
1114721645,
1207340285,
1059690828,
1109451448,
1083205312,
1062563524,
1105831195,
1218565323,
1214232323,
1213682777,
1044259416,
1114606977,
1009257642,
1061334529,
1114048685,
1102739625,
1102740088,
1061162889,
1061163696,
1061163498,
1083128861,
1113047571,
1102718266,
1102739216,
1102744363,
1083207808,
1102739265,
1102739353,
1207192017,
1114606458,
1055404655,
1057958601,
1009257523,
1114143084,
1009254333,
1009255037,
1075215165,
1015842857,
1218565319,
1218565318,
1057959224,
1032975602,
1038122101,
1038122044,
1221768093,
1008212384,
1219786976,
1096367012,
1008283382,
1007349302,
1112444676,
1108958379,
1107943921,
1032976370,
1220878942,
1098324695,
1032976191,
1092118583,
1033126387,
1033233750,
1037879991,
1037912263,
1007193653,
1038107074,
1027555302,
1220054589,
1206675643,
1029886897,
1219087069,
1007729407,
1007348429,
1007316756,
1106739517,
1105109736,
1114606865,
1207829924,
1036588339,
1103583235,
1098324725,
1046694076,
1007335911,
1007334502,
1052371275,
1114015936,
1113953015,
1019273674,
1098052711,
1013209947,
1114650313,
1074595158,
1009520643,
1009439174,
1019273659,
1098324690,
1036251541,
1036251553,
1067404956,
1009520692,
1204953875,
1008605865,
1008293071,
1008378617,
1099862696,
1025895253,
1065975590,
1114606691,
1108933744,
1221234421,
1021226838,
1021226840,
1209298912,
1054831632,
1113411531,
1107198429,
1107940315,
1023369653,
1023370161,
1112989281,
1054830801,
1024591710,
1107941388,
1025439385,
1022854326,
1022743590,
1209933184,
1058928428,
1059543078,
1088354356,
1053591822,
1085401351,
1000006235,
1000476868,
1085422704,
1000473026,
1000473020,
1074914690,
1107938191,
1074227643,
1019986118,
1085401365,
1089272425,
1012895856,
1086272689,
1078493245,
1114134604,
1114134604)""".format(tabel, tabel)
    
    Query_list.append(Query)

In [26]:
df_list = []
        
for query, tabel_name in zip(Query_list, GEODANMARK_60_B):
    print(tabel_name)
    df = pd.read_sql_query(sql=text(query), con=engine.connect())
    
    df['geometri'] = df['geometri'].apply(wkt.loads)
    
    # To GeoDataFrame
    df = gpd.GeoDataFrame(
    df, geometry=df['geometri'])
    
    df.drop(['geometri'], axis=1, inplace=True) # droping geometri so we dont have both geometry and geometri column

    
    print(len(df))
    if len(df) >= 1:
        df_list.append(df)

AFVANDINGSGROEFT
0
ANLAEGDIVERSE
0
BADEBAADEBRO
0
BASSIN
0
BEGRAVELSESOMRAADE
0
BROENDDAEKSEL
0
BRUGSGRAENSE
5
BYGNING
5
BYGVAERK
0
BYKERNE
0
CHIKANE
0
DAEMNING
0
DHMHESTESKO
0
DHMLINJE
0
DIGE
1
ERHVERV
1
FOTOINDEX
0
GARTNERI
0
HAVN
0
HEDE
0
HEGN
9
HELLE
0
HISTORISKFLADE
0
HISTORISKLINJE
0
HISTORISKPUNKT
0
HOEFDE
0
HOEJBEBYGGELSE
0
HOEJSPAENDINGSLEDNING
322
JERNBANE
0
KOMMUNEOMRAADE
0
KRATBEVOKSNING
0
KYST
0
LAVBEBYGGELSE
1
MAST
0
NEDLOEBSRIST
0
OMRAADEPOLYGON
0
PARKERING
0
PARKERINGSOMRAADE
0
PLADS
0
RAASTOFOMRAADE
0
REKREATIVTOMRAADE
0
SANDKLIT
0
SKORSTEN
0
SKOV
78
SKRAENT
7
SOE
0
SPORTSBANE
0
STARTBANE
0
STATUESTEN
0
SYSTEMLINJE
0
TEKNISKANLAEGFLADE
0
TEKNISKANLAEGPUNKT
0
TEKNISKAREAL
1
TELEMAST
0
TOGSTATION
0
TRAE
0
TRAEGRUPPE
0
TRAFIKHEGN
0
UDPEGNINGFLADE
0
UDPEGNINGLINJE
0
UDPEGNINGPUNKT
0
VAADOMRAADE
0
VANDAFSTROEMNINGSOPLAND
0
VANDHAENDELSE
0
VANDKNUDE
0
VANDLOEBSKANT
3
VANDLOEBSMIDTE
4
VEJKANT
11
VEJMIDTE
6
VINDMOELLE
0


In [27]:
Objekter_1 = pd.concat(df_list)

Objekter_1.rename(columns = {'id_lokalid' : 'Objekter_1'}, inplace = True)
Objekter_1.rename(columns = {'tabel' : 'Objekter_1_Tema'}, inplace = True)

Objekter_1.rename(columns = {'geometry' : 'Objekter_1_geometri'}, inplace = True)
#Objekter_1

# Objekter_2 Query

In [28]:
for element in Fejlsteder['Objekter_2'].unique():
    to_query = element + ','
    #print(to_query)

In [29]:
Query_list2 = []
for tabel in GEODANMARK_60_B:
    Query2 = """SELECT '{}' AS TABEL, ID_LOKALID, SDO_UTIL.TO_WKTGEOMETRY(geometri) AS geometri FROM GEODANMARK_60_NOHIST.{}
WHERE
    ID_LOKALID IN (
    1003280860,
1003279847,
1003163957,
1003164046,
1003162043,
1003163522,
1100709652,
1098239398,
1098243177,
1098247798,
1214409791,
1214409801,
1098232981,
1009793884,
1009971263,
1098233257,
1098238467,
1098241496,
1214390701,
1098223328,
1100709648,
1009795178,
1098247790,
1048375687,
1214409792,
1098276814,
1098232970,
1048505223,
1048539784,
1048511232,
1048528260,
1048513810,
1048527882,
1048510578,
1048525810,
1048540625,
1048567962,
1048517466,
1048568510,
1048549336,
1048551303,
1048518985,
1048559820,
1048560890,
1048571804,
1048569191,
1048552556,
1048567861,
1048561158,
1048549247,
1048579923,
1048547510,
1048585200,
1048577530,
1048545371,
1048584871,
1048578391,
1048584661,
1048558483,
1048537488,
1048529729,
1048557067,
1048535985,
1048556330,
1098257436,
1219704044,
1098255986,
1010037887,
1010038577,
1100709660,
1219697509,
1009918206,
1010038583,
1009835785,
1009914805,
1048527715,
1048524845,
1048577875,
1048520674,
1098172628,
1010038014,
1010038439,
1098172867,
1010038108,
1098245956,
1098245846,
1214390700,
1214390699,
1009971202,
1208533821,
1048532387,
1214390698,
1098257154,
1009971298,
1208973337,
1212099056,
1003282139,
1003282421,
1039781761,
1114184423,
1209948296,
1003280407,
1003279923,
1114184461,
1219748785,
1003165703,
1003165643,
1003163728,
1003165712,
1003165715,
1003164026,
1003165738,
1114146133,
1003162506,
1101290783,
1219614202,
1054703731,
1009938226,
1009940101,
1009971213,
1009938451,
1098269088,
1009938489,
1009934023,
1009938428,
1040098178,
1040259080,
1040172011,
1040257283,
1097210341,
1003282096,
1111951547,
1003281628,
1003165378,
1003165413,
1003166244,
1003165443,
1003165425,
1003163188,
1099982588,
1054761466,
1099131461,
1099034455,
1054761468,
1054761489,
1099029661,
1211636866,
1099091180,
1098980295,
1098979486,
1099982168,
1099093630,
1099090108,
1054654318,
1054658030,
1009970568,
1098308982,
1009968255,
1049509150,
1114674038,
1006722765,
1040123987,
1097206828,
1054654273,
1054659043,
1098984092,
1099113115,
1096021979,
1039394228,
1098174477,
1006722775,
1006728238,
1219452582,
1006722762,
1006722757,
1006722746,
1006642013,
1006728062,
1006711634,
1006711632,
1006711640,
1006711651,
1096330754,
1003287833,
1003287637,
1003164296,
1003166212,
1003166154,
1061620360,
1061659919,
1213682797,
1055844178,
1213688455,
1213688454,
1055887859,
1114185024,
1039077385,
1096330734,
1006722538,
1006722529,
1006722524,
1114042110,
1003285757,
1003285805,
1003286088,
1003287568,
1003286248,
1003286854,
1003287634,
1207422539,
1056055452,
1114042126,
1061671729,
1061646602,
1054426520,
1055363700,
1057338525,
1003287517,
1003287062,
1003287506,
1003287444,
1003287621,
1054411343,
1221407207,
1055871656,
1055729390,
1055843463,
1091440745,
1055497984,
1055827802,
1055843182,
1055841930,
1055363467,
1055498123,
1213670884,
1055495415,
1217702489,
1091518663,
1091518676,
1044640935,
1055363655,
1044680298,
1044035406,
1093388986,
1057317680,
1108226839,
1003285700,
1114028713,
1207340276,
1222463446,
1037728409,
1009255235,
1222463412,
1083206194,
1083206276,
1214232323,
1044669748,
1043886894,
1044737571,
1044688681,
1209786204,
1009255137,
1092173735,
1092173701,
1222463432,
1222463429,
1009254563,
1061229505,
1061204170,
1219434400,
1102678655,
1061163492,
1061163920,
1061163491,
1083206188,
1061163484,
1083087264,
1102739625,
1061338524,
1059599264,
1017854304,
1219357074,
1055406738,
1057958601,
1010340035,
1076057775,
1007335930,
1098792515,
1009188784,
1113887988,
1009254333,
1009255037,
1098789024,
1222463448,
1092259142,
1043847553,
1098791481,
1007656904,
1007656903,
1007349389,
1114717833,
1033222094,
1220878942,
1217660781,
1037889269,
1037951787,
1008212384,
1008283383,
1207457204,
1008286785,
1075215165,
1114054714,
1007231486,
1007229037,
1114054746,
1007231414,
1007349302,
1219087217,
1112569457,
1218392566,
1019032692,
1019265214,
1105299567,
1114671423,
1018943483,
1013632880,
1032976370,
1208124717,
1114717843,
1033126387,
1033236584,
1033326269,
1032976191,
1049091161,
1206192131,
1071721797,
1007190160,
1047851388,
1007193653,
1027913596,
1029396428,
1075215120,
1220073534,
1114054730,
1007671235,
1007044006,
1076057771,
1114054741,
1114713004,
1219087178,
1017082592,
1219087069,
1019009434,
1019040243,
1018440144,
1015586552,
1107034754,
1036593543,
1219794143,
1103583235,
1219794148,
1033116482,
1036623377,
1095070632,
1213002679,
1114029331,
1033091523,
1033116461,
1007191800,
1210532395,
1220065017,
1220065015,
1029496714,
1007325556,
1113953015,
1017106313,
1013209947,
1033995421,
1036097782,
1033960392,
1050090994,
1074595158,
1009439174,
1009491348,
1009438689,
1050060972,
1008305314,
1008297024,
1008375109,
1204673862,
1015594923,
1016463066,
1015597067,
1036881153,
1036752887,
1034609407,
1033920153,
1210702641,
1216242333,
1034622288,
1009441822,
1009491381,
1009520692,
1009520643,
1099911973,
1220167449,
1204673845,
1048793685,
1008378532,
1037031410,
1036872092,
1036208787,
1033870033,
1033892330,
1008378617,
1008374278,
1008376947,
1009491380,
1008293071,
1008292995,
1088197182,
1008378515,
1204673802,
1008374905,
1221676841,
1025909159,
1025999469,
1025998247,
1025895253,
1025998570,
1042884327,
1043106236,
1113324302,
1042978667,
1066122710,
1024591642,
1071782291,
1111434819,
1023370164,
1023453015,
1023356713,
1023458306,
1023458309,
1112327199,
1112300742,
1023369642,
1023357912,
1021170045,
1021164184,
1088027677,
1111436689,
1024543326,
1110146362,
1113414548,
1221074951,
1019741554,
1019734006,
1019701520,
1221241398,
1110122127,
1088195225,
1110097896,
1088195313,
1110146329,
1022854326,
1022873237,
1022855442,
1022740680,
1022740836,
1022747751,
1022858877,
1221241394,
1209214507,
1012752158,
1022740875,
1022859838,
1022677215,
1025233737,
1025236456,
1024761347,
1024753592,
1110450663,
1000117838,
1085341513,
1000169619,
1085344663,
1000003667,
1000019999,
1000476725,
1000121913,
1085422704,
1000165060,
1000165234,
1000004417,
1105706505,
1085343008,
1085343050,
1214861776,
1085420113,
1085324920,
1085323044,
1085335874,
1105705438,
1209932636,
1209932875,
1201733714,
1000122182,
1000005965,
1085401316,
1000165363,
1219990740,
1000473020,
1081862323,
1087081542,
1087064284,
1026752815,
1022856414,
1000121899,
1101441545,
1101442483,
1085355731,
1085377868,
1085382512,
1085382502,
1085373792,
1085374097,
1085369279,
1085390198,
1085385644,
1085398764,
1222423260,
1203788112,
1012263556,
1011369992,
1074227643,
1074227649,
1011370186,
1011371865,
1086272676,
1086272660,
1012895856,
1086272685,
1012893182,
1011833850,
1078493250,
1221052836,
1078493245,
1208756374,
1208744331,
1031544911,
1031543742,
1031543990,
1031439713,
1031544556,
1031434251,
1031420733,
1031439708,
1031542349,
1031462747,
1114134604,
1114573318,
1207081340,
1113476608,
1003261107,
1107942910,
1040905769,
1040835534,
1040877649,
1040905815,
1205545285,
1002620233,
1214409802,
1009713256,
1009713266,
1009713238,
1009713604,
1009915650,
1009821390,
1098167019,
1009914798,
1003279947,
1003280839,
1013982201,
1098266252,
1108189127,
1098167021,
1050131678,
1040260080,
1212095582,
1203792676,
1039837720,
1003261156,
1003261135,
1037318040,
1003261103,
1003261142,
1114714235,
1099111548,
1216643169,
1058683854,
1216642078,
1058770694,
1216641893,
1216641920,
1216643171,
1058678041,
1216642002,
1058686265,
1099107907,
1216641825,
1216643159,
1216643163,
1216643170,
1058774480,
1211632025,
1099111552,
1216643162,
1114647340,
1202210777,
1102475585,
1009995619,
1009995604,
1009684457,
1040230226,
1096022113,
1214923335,
1003261151,
1003261125,
1041823725,
1094761786,
1041914107,
1107943842,
1100086862,
1211312791,
1041850051,
1096021635,
1041850053,
1214923339,
1003261126,
1062563535,
1061697066,
1091488946,
1091489262,
1114587207,
1055401637,
1108481930,
1055366045,
1057338706,
1093377286,
1093377287,
1044803024,
1061710430,
1015721292,
1055820780,
1055848036,
1055820851,
1091518544,
1055729261,
1213688969,
1107943908,
1213682703,
1213688968,
1108217193,
1055404650,
1055296633,
1091515884,
1055363634,
1107943779,
1213682777,
1114647844,
1114606603,
1114647843,
1114647119,
1108217077,
1114721649,
1114713669,
1207340285,
1059691201,
1059690828,
1114650070,
1016307557,
1019275574,
1015842857,
1218565330,
1055404656,
1059167074,
1114647206,
1044259416,
1090016145,
1040579136,
1114713665,
1061163696,
1061163498,
1061162889,
1114048685,
1102739353,
1083207808,
1102739265,
1083205314,
1207192017,
1114606454,
1102739216,
1083128861,
1102718266,
1102744363,
1113047571,
1102740088,
1062563542,
1062563524,
1097576247,
1057985288,
1009257642,
1113242863,
1009257523,
1114143084,
1062563537,
1105831195,
1218565321,
1107943921,
1098324755,
1037879991,
1037954187,
1037954565,
1037912263,
1221768093,
1038107074,
1008283382,
1096367012,
1219786976,
1114713671,
1114713670,
1114662265,
1105109737,
1108958378,
1057959224,
1033233750,
1098324699,
1033622337,
1037954280,
1006983406,
1038122044,
1027515079,
1027515096,
1029400122,
1220054589,
1219438369,
1108189800,
1007348429,
1007154748,
1007316756,
1114662264,
1114645301,
1112444661,
1112444667,
1107148772,
1107148773,
1091270373,
1036588339,
1098324591,
1098325473,
1046694074,
1046694072,
1098052581,
1052328429,
1007334502,
1007335911,
1098052578,
1016437365,
1098052712,
1019273645,
1098324730,
1098324783,
1046694075,
1098324718,
1008605860,
1204953875,
1008605859,
1008605871,
1112444676,
1207829924,
1036251541,
1098324782,
1036251553,
1098324690,
1033922892,
1098324685,
1008605865,
1008605861,
1019275556,
1098052711,
1098324692,
1008409362,
1021556987,
1025982067,
1025982079,
1107940321,
1026002155,
1065976634,
1025982084,
1065974838,
1021163318,
1021236217,
1021226942,
1030681042,
1065975590,
1221234421,
1060641355,
1107940271,
1021234203,
1024591710,
1054830801,
1107940188,
1060641357,
1021233214,
1065866798,
1058928428,
1114609039,
1022743590,
1066205768,
1110454397,
1110454396,
1107940294,
1209933184,
1107894230,
1000476868,
1085401351,
1000006235,
1000473026,
1114609025,
1113750464,
1091276778,
1019986118,
1085401365,
1085398602,
1107938145,
1089272425,
1107893547,
1074914690,
1086272689,
1107196733,
1107148664,
1113761503,
1031581783)""".format(tabel, tabel)
    
    Query_list2.append(Query2)

In [30]:
df_list2 = []
        
for query, tabel_name in zip(Query_list2, GEODANMARK_60_B):
    print(tabel_name)
    df = pd.read_sql_query(sql=text(query), con=engine.connect())
    
    df['geometri'] = df['geometri'].apply(wkt.loads)
    
    # To GeoDataFrame
    df = gpd.GeoDataFrame(
    df, geometry=df['geometri'])
    
    df.drop(['geometri'], axis=1, inplace=True) # droping geometri so we dont have both geometry and geometri column

    
    print(len(df))
    if len(df) >= 1:
        df_list2.append(df)

AFVANDINGSGROEFT
1
ANLAEGDIVERSE
0
BADEBAADEBRO
0
BASSIN
0
BEGRAVELSESOMRAADE
0
BROENDDAEKSEL
0
BRUGSGRAENSE
39
BYGNING
17
BYGVAERK
0
BYKERNE
0
CHIKANE
0
DAEMNING
0
DHMHESTESKO
0
DHMLINJE
0
DIGE
1
ERHVERV
1
FOTOINDEX
0
GARTNERI
0
HAVN
0
HEDE
0
HEGN
78
HELLE
0
HISTORISKFLADE
0
HISTORISKLINJE
0
HISTORISKPUNKT
0
HOEFDE
0
HOEJBEBYGGELSE
1
HOEJSPAENDINGSLEDNING
172
JERNBANE
0
KOMMUNEOMRAADE
0
KRATBEVOKSNING
0
KYST
0
LAVBEBYGGELSE
5
MAST
0
NEDLOEBSRIST
0
OMRAADEPOLYGON
0
PARKERING
2
PARKERINGSOMRAADE
0
PLADS
0
RAASTOFOMRAADE
0
REKREATIVTOMRAADE
0
SANDKLIT
0
SKORSTEN
0
SKOV
368
SKRAENT
16
SOE
0
SPORTSBANE
0
STARTBANE
0
STATUESTEN
0
SYSTEMLINJE
0
TEKNISKANLAEGFLADE
1
TEKNISKANLAEGPUNKT
0
TEKNISKAREAL
4
TELEMAST
0
TOGSTATION
0
TRAE
0
TRAEGRUPPE
0
TRAFIKHEGN
0
UDPEGNINGFLADE
0
UDPEGNINGLINJE
0
UDPEGNINGPUNKT
0
VAADOMRAADE
2
VANDAFSTROEMNINGSOPLAND
0
VANDHAENDELSE
0
VANDKNUDE
0
VANDLOEBSKANT
11
VANDLOEBSMIDTE
13
VEJKANT
34
VEJMIDTE
31
VINDMOELLE
0


In [31]:
Objekter_2 = pd.concat(df_list2)

Objekter_2.rename(columns = {'id_lokalid' : 'Objekter_2'}, inplace = True)
Objekter_2.rename(columns = {'tabel' : 'Objekter_2_Tema'}, inplace = True)

Objekter_2.rename(columns = {'geometry' : 'Objekter_2_geometri'}, inplace = True)
#Objekter_2

# Højspænding skal krydse > 5 meter over andre objekter !
Vi ændre de objekter som løber under Højspænding

In [33]:
# Merging our DataFrames!


result = pd.merge(Fejlsteder, Objekter_1, on=['Objekter_1'], how='left')

result_1 = pd.merge(result, Objekter_2, on=['Objekter_2'], how='left')
print(len(result_1))



1638


# Sorter i Hoejspaendingsledning og objekterne som løber under

In [34]:
# Drop rows with Nan values
result_1 = result_1.dropna()
print(len(result_1))

1395


In [35]:
# Hoejspaending

hoej = []
hoej_geom = []
hoej_id_lokalid = []


for i, row in result_1.iterrows():
    if (row['Objekter_1_Tema'] == 'HOEJSPAENDINGSLEDNING'):
        
        hoej.append(row['Objekter_1_Tema'])
        
        hoej_geom.append(row['Objekter_1_geometri'])
        
        hoej_id_lokalid.append(row['Objekter_1'])
        
    elif (row['Objekter_2_Tema'] == 'HOEJSPAENDINGSLEDNING'):
        
        hoej.append(row['Objekter_2_Tema'])
        
        hoej_geom.append(row['Objekter_2_geometri'])
        
        hoej_id_lokalid.append(row['Objekter_2'])
        

        
result_1['HOEJ'] = hoej

result_1['HOEJ_Geom'] = hoej_geom

result_1['HOEJ_id_lokalid'] = hoej_id_lokalid 

In [36]:
# Objekter

objekt = []
objekt_geom = []
objekt_id_lokalid = []


for i, row in result_1.iterrows():
    if (row['Objekter_1_Tema'] != 'HOEJSPAENDINGSLEDNING'):
        
        objekt.append(row['Objekter_1_Tema'])
        
        objekt_geom.append(row['Objekter_1_geometri'])
        
        objekt_id_lokalid.append(row['Objekter_1'])
        
    elif (row['Objekter_2_Tema'] != 'HOEJSPAENDINGSLEDNING'):
        
        objekt.append(row['Objekter_2_Tema'])
        
        objekt_geom.append(row['Objekter_2_geometri'])
        
        objekt_id_lokalid.append(row['Objekter_2'])
    
    # If both objects are Hoejspaending, take the Hoejspaending that is not in 'HOEJ_id_lokalid'
    elif (row['HOEJ_id_lokalid'] != row['Objekter_1']):
        
        objekt.append(row['Objekter_1_Tema'])
        
        objekt_geom.append(row['Objekter_1_geometri'])
        
        objekt_id_lokalid.append(row['Objekter_1'])
        
    elif (row['HOEJ_id_lokalid'] != row['Objekter_2']):
        
        objekt.append(row['Objekter_2_Tema'])
        
        objekt_geom.append(row['Objekter_2_geometri'])
        
        objekt_id_lokalid.append(row['Objekter_2'])
        
           
result_1['objekt'] = objekt

result_1['objekt_geom'] = objekt_geom

result_1['objekt_id_lokalid'] = objekt_id_lokalid

In [37]:
# Drop columns we are not going to use
result_1 = result_1.drop(['Objekter_1', 'Objekter_2', 'Objekter_1_Tema', 'Objekter_1_geometri', 'Objekter_2_Tema', 'Objekter_2_geometri'], axis=1)

In [38]:
# buffer the Points in the collumn Sted
result_1['Sted'] = result_1['Sted'].astype(str)

result_1['Sted'] = result_1['Sted'].apply(wkt.loads)

In [40]:
# Get a column with geometry type from the objekt column
result_1 = result_1.set_geometry('objekt_geom')

# Listing geometry Type
result_1['geom_type_objekt'] = result_1.geom_type

print(len(result_1))

1395


In [41]:
# Filter for 'Højspænding skal krydse > 5 meter over alle objekter'
result_1 = result_1.loc[result_1['tekst'] == 'Højspænding skal krydse > 5 meter over alle objekter'].copy()

print('Vi har i alt :',len(result_1), 'Objekter som skal rettes ved Reglen --> Højspænding skal krydse > 5 meter over alle objekter dog gentager nogle af disse objekter sig i dataframe fordi objektet har flere punkter som krydser' )

Vi har i alt : 812 Objekter som skal rettes ved Reglen --> Højspænding skal krydse > 5 meter over alle objekter dog gentager nogle af disse objekter sig i dataframe fordi objektet har flere punkter som krydser


## Test of a single object

In [42]:
# Hvis Vi vil teste Et Objekt !
#result_1 = result_1.loc[result_1['objekt_id_lokalid'] == '1110146362'].copy()

In [43]:
#with pd.option_context("display.max_rows", None):
#    display(result_1)

# Coord List of the Objects

In [44]:
def give_coordlist(x, y):
    # Returns a list of tupels with (x,y,z) coordinates
    if y == 'LineString':
        
        coord_list = list(shape(x).coords) # VI ER HER
        
        interior_list = []
        
        return coord_list, interior_list,
    
    elif y == 'Polygon':
        
        coord_list = list(x.exterior.coords) # Exterior Coords
        
        interior_coords = [] # Interior_coords
        
        if (len(x.interiors) > 0):
        
            for interior in x.interiors:
                #interior_coords += interior.coords[:]
                interior_coords.append(interior.coords[:])
            return coord_list, interior_coords
            
        else:
            return coord_list, interior_coords,
    

result_1['coord_list_objects'], result_1['coord_list_objects_interior']  = zip(*result_1.apply(lambda x: give_coordlist(x['objekt_geom'],x['geom_type_objekt']),axis=1))

In [45]:
# Length of the coord list or number of points in the orginal geometry
result_1['coord_list_length'] = result_1['coord_list_objects'].map(len)

# Number Of Interior Polygons
result_1['NumberOfInteriorPolygons'] = result_1['coord_list_objects_interior'].map(len)

# Reset index
result_1.reset_index(drop=True, inplace=True)

print(len(result_1))

812


In [46]:
#with pd.option_context("display.max_rows", None):
#    display(result_1)

# ----- Exterior Polygons -----

## Exterior, Coord List to Points

In [47]:
new_points = []

for i, row in result_1.iterrows():  # for index, row in dataframe
    
    # Hoejspænding geometri
    hoej_geom = row['HOEJ_Geom']
    
    
    coord_list = row['coord_list_objects']
    
    id_lokalid = row['objekt_id_lokalid']
    
    tabel = row['objekt']
    
    geom_type = row['geom_type_objekt']
    
    tekst = row['tekst']
    
    størrelse = row['størrelse']
    
    index = i
    
    sted = row['Sted']
    
    number_of_points = row['coord_list_length']
    
    NumberOfInteriorPolygons = row['NumberOfInteriorPolygons']
    
    if geom_type != None:
        for i, coords in enumerate(row['coord_list_objects']): # for index, coords in row['coord_list']
            #print(index)
            #print(coords)
            point = shapely.geometry.Point(coords) # Create point from (x, y, z) Coords 
            #print(point)
            new = [point, id_lokalid, i, tabel, geom_type, tekst, størrelse, index, hoej_geom, sted, number_of_points, NumberOfInteriorPolygons] # add the point,id_lokalid and the index of the point in the linestring to a list
            new_points.append(new)
        
# To Pandas DataFrame
data_points = pd.DataFrame(new_points)

# Renaming columns
data_points.rename(columns = {0 : 'geometry', 1 : 'id_lokalid', 2 : 'id_point_list', 3 : 'tabel', 4 : 'geom_type', 5 : 'tekst', 6 : 'Størrelse', 7 : 'index', 8 : 'hoej_geom', 9 : 'sted', 10 : 'Number_of_points', 11 : 'NumberOfInteriorPolygons'}, inplace = True)

# To GeoDataFrame
data_points = gpd.GeoDataFrame(
    data_points, geometry=data_points['geometry'])

# Add new column 
data_points['Exterior'] = 'Yes'

# Sum of coord_list_length should be equal to length of the dataframe 'data_points'
assert len(data_points) == result_1['coord_list_length'].sum(), "Sum of coord_list_length should be equal to length of the dataframe 'data_points'"

#checking how many points we have
print(len(data_points))

182493


## Exterior, Points To LineSegments and Check if Object crosses Hoejspaending

In [48]:
# Get x, y, z coords
data_points['x_coord'] = data_points['geometry'].x
data_points['y_coord'] = data_points['geometry'].y
data_points['z_coord'] = data_points['geometry'].z

In [49]:
# Groupby index since some objects have multiple entities
data_points_group = data_points.groupby('index').agg(lambda x: list(x))

assert len(data_points_group) == len(result_1), 'These dataframes should have same length'

In [50]:
from shapely.geometry import Point, LineString

change_points = []

# Points To LineSegments
for i, row in data_points_group.iterrows():
   
    counter = 1

    # Sted to GeoSeries !
    sted_geom = geopandas.GeoSeries(row['sted'][0])
    sted_geom = sted_geom.set_crs("EPSG:25832")
    sted_geom = sted_geom.buffer(5)
    
        
    for point, id_point_list, Z_Størrelse in zip(row['geometry'], row['id_point_list'], row['Størrelse']):
        
        
        # We have to break the loop when we run out of points!
        if counter > row['id_point_list'][-1]:
            break
            
        else:
            
            # The counter is used to pick the point in front of the current point, So that we can make a LineString of the current point and the next point in the list 
            Line = LineString([point, row['geometry'][counter]])
                
            # The line we just created to GeoSeries
            Line_GeoSeries = geopandas.GeoSeries(Line)
            Line_GeoSeries = Line_GeoSeries.set_crs("EPSG:25832")
            
            # Add one to counter
            counter = counter + 1
            
            # check if our Linesegment crosses the buffed point sted_geom
            for _bool in Line_GeoSeries.crosses(sted_geom, align=True):
                if (_bool == True):
                    new2 = [i, row['id_lokalid'][0], id_point_list, row['id_point_list'][counter - 1], Z_Størrelse]
                    change_points.append(new2)

## Exterior, Bool_crossing Dataframe

In [51]:
# To Pandas DataFrame
Bool_crossing = pd.DataFrame(change_points)

# Renaming columns
Bool_crossing.rename(columns = {0 : 'Index', 1 : 'objekt_id_lokalid', 2 : 'id_point_list_first_point', 3 : 'id_point_list_secound_point', 4 : 'Z_Størrelse'}, inplace = True)

In [52]:
# Handeling Z_størrelse where its written in cm, if the diffrence is less than 1m its written in centimeters so we are just converting it to centimeters. Bigger than five is because then it must be written in CM!
Bool_crossing['Z'] = Bool_crossing['Z_Størrelse'].map(lambda x: x.lstrip('Z difference: ').rstrip('cm'))

Bool_crossing['Z'] = Bool_crossing['Z'].str.replace(',', '.')

Bool_crossing['Z'] = Bool_crossing['Z'].astype(float)

Bool_crossing['Z'] =  np.where(Bool_crossing['Z'] > 5, Bool_crossing['Z'] / 100, Bool_crossing['Z'])

In [53]:
#Bool_crossing_test = Bool_crossing.loc[Bool_crossing['objekt_id_lokalid'] == '1110146362']

In [54]:
#with pd.option_context("display.max_rows", None):
#    display(Bool_crossing_test)

## Exterior, Merge Bool_crossing with data_points_group

In [55]:
# Groupby index
Bool_crossing_group = Bool_crossing.groupby('Index').agg(lambda x: list(x))

In [56]:
done = Bool_crossing_group.join(data_points_group)

print(len(done))

770


In [57]:
# Subtract the minimum value in the list column by 1 and add it to the list
done['id_point_list_first_point'] = done['id_point_list_first_point'].apply(lambda x: x + [min(x) - 1])

# plus the max value in the list column by 1 and add it to the list
done['id_point_list_secound_point'] = done['id_point_list_secound_point'].apply(lambda x: x + [max(x) + 1])

## Exterior, Lodrette Vektore

In [58]:
def zero_with_max(lst, max_val):
    if 0 in lst:
        lst.append(max_val)
    return lst

def max_with_zero(lst, max_val):
    if max_val in lst:
        lst.append(0)
    return lst

In [59]:
done['Max_Value'] = done['id_point_list'].apply(lambda x: max(x))

done['id_point_list_first_point'] = done.apply(lambda row: zero_with_max(row['id_point_list_first_point'], row['Max_Value']), axis=1)

done['id_point_list_secound_point'] = done.apply(lambda row: max_with_zero(row['id_point_list_secound_point'], row['Max_Value']), axis=1)

## Exterior, Edit the Z Coord

In [60]:
change_z = []

for i, row in done.iterrows():
    
    for id_point, x, y, z, geom in zip(row['id_point_list'], row['x_coord'], row['y_coord'], row['z_coord'], row['geometry']):
        if (id_point in row['id_point_list_first_point'] or id_point in row['id_point_list_secound_point']):
            new_z = max(round(z + row['Z'][0] - 10, 2), 2)
            coords = (x, y, new_z)
            point = Point(coords)
            #print('old point ;', geom)
            #print('new point ;', point)
            changed = 'Yes'
            new_changed_z = [i, id_point, point, changed, row['tabel'][0], row['geom_type'][0], row['id_lokalid'][0], new_z]
            change_z.append(new_changed_z)
            #print('Id_point_list; ', id_point, 'Z_coord; ', z, '-------->', row['Z'][0] - 5.5, 'New Z : ', new_z)
        else:
            changed = 'No'
            not_change_z = [i, id_point, geom, changed, row['tabel'][0], row['geom_type'][0], row['id_lokalid'][0], z]
            change_z.append(not_change_z)
            
       
# To Pandas DataFrame
edit_z = pd.DataFrame(change_z)

# Renaming columns
edit_z.rename(columns = {0 : 'Index', 1 : 'id_point_list', 2 : 'geometry', 3 : 'Changed', 4 : 'tabel', 5 : 'geom_type', 6 : 'Id_Lokalid', 7 : 'Z'}, inplace = True)            

# WKT
#edit_z['geometry'] = edit_z['geometry'].apply(wkt.loads)

# To GeoDataFrame
edit_z = gpd.GeoDataFrame(
    edit_z, geometry=edit_z['geometry'])
    

In [61]:
# Groupby index
edit_z_group = edit_z.groupby('Index').agg(lambda x: list(x))
edit_z_group.reset_index(inplace=True)

In [62]:
print(len(edit_z_group))

770


## Exterior, handle objects who have multiple entries

In [66]:
edit_z_group['id_lokalid'] = edit_z_group['Id_Lokalid'].map(lambda x: x[0])

In [67]:
# Groupby 'id_lokalid'
grouped = edit_z_group.groupby('id_lokalid').agg(lambda x: list(x))

In [68]:
print('Der Findes i alt', len(edit_z_group), 'Punkter som krydser objekterne dog findes der kun', len(grouped), 'unikke objekter så nogle objekter bliver krydset flere steder, disser har multiple entries')

Der Findes i alt 770 Punkter som krydser objekterne dog findes der kun 482 unikke objekter så nogle objekter bliver krydset flere steder, disser har multiple entries


### Function 'create_namedtuple'
The create_namedtuple function takes four arguments: name, which is the name of the named tuple class, fields, which is a list of field names, values, which is a list of values, and fillvalue, which is the value to use for filling in missing values (None by default). The function creates a new class called name that inherits from collections.namedtuple and has the field names specified in fields.

The function uses the zip_longest function to iterate over the fields and values lists in parallel and automatically fills in missing values with fillvalue. And then use the dictionary to instantiate the named tuple.

In [69]:
from collections import namedtuple
from itertools import zip_longest

# The function is called in the for loop below !
def create_namedtuple(name, fields, values, fillvalue):
    TupleType = namedtuple(name, fields)
    data = zip_longest(fields, values, fillvalue=fillvalue)
    lists = [item[1] for item in TupleType(*data)]
              
    return lists

### For Loop
This code is using a for loop to iterate through the rows of a Dataframe called "grouped". It is checking if each row has more than one element in the 'Index' column, and if so, it is using the 'create_namedtuple' function to create named tuples of equal length for the 'Changed', 'geometry', and 'id_point_list' columns. It then uses another for loop to iterate through the elements of these named tuples and performs some logic to check if there are any changed points in the 'Changed' entries and if so, it gets the index of the changed points and gets the corresponding points from the 'geometry' entries and finds the point with the smallest Z value. It then appends the id_lokalid, id_point_list and this minimum Z point to a list called 'done' and adds 'done' to a list called 'done_done' outside of the for loop. If the original row from the 'grouped' dataframe only had one element in the 'Index' column, it appends the row directly to 'done_done' outside the for loop.

In [70]:
done_done = []

# IF some objects have more than 10 Entities update the 'fields' List with more fields ect.. list11, list12 !! 
fields = ['list1', 'list2', 'list3', 'list4', 'list5', 'list6', 'list7', 'list8', 'list9', 'list10']


# The Dataframe grouped have some collumns with neasted list's (list's inside a list)
# The problem is that our neasted list's have diffrent length's therfore we have to use the function 'create_namedtuple'
# To create List that have equal lenght.
for q, row in grouped.iterrows():
    
    #print('---------------', len(row['Index']), '--------------')
    #print('id_lokalid ;', q)
    #print(row['id_point_list'][0])
    
    geom_type = row['geom_type'][0]
    
    tabel = row['tabel'][0]
    
    #print(tabel)
    
    
    if (len(row['Index']) > 1):
        # If the object have multiple entities
        
        #print(len(row['Index']))
        
        
        # fillvalue is used in our function 'create_namedtuple'
        n  = len(row['Changed'][0])
        
        
        fillvalue = ['X' for x in range(n)] 
        
        # create_namedtuple for row['Changed']
        Changed = create_namedtuple('MyTuple_Changed', fields, row['Changed'], fillvalue)
        
        # create_namedtuple for row['geometry']
        Points = create_namedtuple('MyTuple_Points', fields, row['geometry'], fillvalue)
        
        # create_namedtuple for row['id_point_list'] 
        id_point_list = create_namedtuple('MyTuple_id_point_list', fields, row['id_point_list'], fillvalue)
        
        
        
        # used to append the list done to the list done_done outside the for loop
        #counter = 2
        
        # List outside the Neasted for loop
        done = []
        

        # for elements in all the Named Tupels
        for a, b, c, d, e, f, g, h, i, j, a1, b1, c1, d1, e1, f1, g1, h1, i1, j1, a2, b2, c2, d2, e2, f2, g2, h2, i2, j2 in zip(*Changed, *Points, *id_point_list):
            
   
            # Changed to list
            Changed_entries = [a, b, c, d, e, f, g, h, i, j]
            
            # Points to list
            Points_entries = [a1, b1, c1, d1, e1, f1, g1, h1, i1, j1]
            
            # Id_point_list
            id_point_list_entries = [a2, b2, c2, d2, e2, f2, g2, h2, i2, j2]
            
            #print(Changed_entries)
            #print(id_point_list_entries)
            
            
            
            # if 'Yes' in Changed_entries (there are changed points in the list 'Points_entries')
            if ('Yes' in Changed_entries):
                
                
                #print(Changed_entries)
                #print(id_point_list_entries)
                
                # Getting the index of the changed points, since these equals the index of 'Yes' in the list 'index_of_yes'
                index_of_yes = [i for i, v in enumerate(Changed_entries) if v == 'Yes']
                #print(f'Index of Yes: {index_of_yes}')
                
                
                # Getting only the changed Points in a seprate list 'result'
                ids = index_of_yes
                values = Points_entries
                
                # Logic ---> value for index, value in enumarate(list1) if index in list2
                result = [val for i, val in enumerate(values) if i in ids]
                
                #print(result)
                
                # Getting the Point with the smallest Z value from the list of points 'result'
                min_z_point = min(result, key=lambda p: p.z)
                
                #print('Min z point :', min_z_point)
                
                # Append the id_lokalid, id_point_list and minimum z coord point to the list 'done'
                row_done = [q, id_point_list_entries[0], min_z_point, geom_type[0], tabel[0]]
                done.append(row_done)
                
                #print(done)
                #print(len(done), 'lenght of done')
                
                # Appending the list done to the list done_done if counter equal len
                if (len(done) == n):
                     # To Pandas DataFrame
                    done_pd = pd.DataFrame(done)
                    
                    # Renaming columns
                    done_pd.rename(columns = {0 : 'Id_Lokalid', 1 : 'id_point_list', 2 : 'geometry', 3: 'geom_type', 4 : 'tabel'}, inplace = True)
                    
                    #print(done_pd)
                    
                  
                    #print(len(done), n, 'append outside loop, yes in changed')
                    done_done.append(done_pd)
 
            else:
                # Append the id_lokalid, id_point_list and point to the list 'done'
                row_done = [q, id_point_list_entries[0], Points_entries[0], geom_type[0], tabel[0]]
                
                done.append(row_done)

                if (len(done) == n):
                    
                    # To Pandas DataFrame
                    done_pd = pd.DataFrame(done)

                    # Renaming columns
                    done_pd.rename(columns = {0 : 'Id_Lokalid', 1 : 'id_point_list', 2 : 'geometry', 3: 'geom_type', 4 : 'tabel'}, inplace = True) 
                    
                    #print(done_pd)
                    
                    
                    #print(len(done), n, 'append outside loop, not yes in changed')
                    done_done.append(done_pd)
                    
    else:
        
        done = []
        
        #print('ONLY ONE ENTITY')
        
        for a, b in zip(row['id_point_list'][0], row['geometry'][0]):

            row_done = [q, a, b, geom_type[0], tabel[0]]

            done.append(row_done)

            if(len(row['Changed'][0]) == len(done)):
                
                
                
                # To Pandas DataFrame
                done_pd = pd.DataFrame(done)

                # Renaming columns
                done_pd.rename(columns = {0 : 'Id_Lokalid', 1 : 'id_point_list', 2 : 'geometry', 3 : 'geom_type', 4 : 'tabel'}, inplace = True)

                #print(done_pd)

                #print(len(done), len(row['Changed'][0]), 'append outside loop, only one entity')
                #print(done_pd)

                done_done.append(done_pd)

In [71]:
# To Pandas DataFrame
multiple_entries = pd.concat(done_done)

# group by id_lokalid
multiple_entries_group = multiple_entries.groupby('Id_Lokalid').agg(lambda x: list(x))

assert len(multiple_entries_group) == len(grouped), 'These dataframes should have same length'

In [72]:
# Checking if id_point_list is sorted right!

for i, row in  multiple_entries_group.iterrows():
    counter = 0
    for element, nxt in zip(row['id_point_list'], row['id_point_list'][1:]):
        while counter == 0:
            if (counter != element):
                print(counter, element)
            break
        counter = counter + 1
        if (counter != nxt):
            print(counter, nxt)
        

### Test for 'lodret vektor'

In [73]:
#multiple_entries_group_test = multiple_entries_group.loc[multiple_entries_group['Id_Lokalid'] == '1036097782']

In [74]:
#for i, row in  multiple_entries_group_test.iterrows():
#     for i, value in enumerate(row['geometry']):
#            if i == 0 or i == len(row['id_point_list']) - 1:
#                print(i, value)
#x = []
#y = []
#z = []

#for i, row in  multiple_entries_group_test.iterrows():
#     for i, value in enumerate(row['geometry']):
#            x.append(value.x)
#            y.append(value.y)
#            z.append(value.z)
            
#df = pd.DataFrame({'x': x, 'y': y, 'z': z})

In [75]:
#grouped = df.groupby(['x', 'y'])['z'].apply(lambda x: list(x))

In [76]:
#with pd.option_context("display.max_rows", None):
#    display(grouped)

# ----- Interior Polygons -----

## Interior, Coord List to Points

In [77]:
new_points = []


for i, row in result_1.iterrows():  # for index, row in dataframe
    if len(row['coord_list_objects_interior']) > 0:
   
        hoej_geom = row['HOEJ_Geom']

        id_lokalid = row['objekt_id_lokalid']

        tabel = row['objekt']

        geom_type = row['geom_type_objekt']

        tekst = row['tekst']

        størrelse = row['størrelse']

        index = i

        sted = row['Sted']

        TotalNumberOfInteriorPolygons = row['NumberOfInteriorPolygons']
        
        #print(len(row['coord_list_objects_interior']))
        
        for NumberOfInteriorPolygon, coord_list in enumerate(row['coord_list_objects_interior']):
            
            #print(len(coord_list))
            #print(coord_list)
            
            NumberOfInteriorPolygon = NumberOfInteriorPolygon + 1
            
            number_of_points = len(coord_list)
            
            for i, coords in enumerate(coord_list):
                point = shapely.geometry.Point(coords) # Create point from (x, y, z) Coords 
                new = [point, id_lokalid, i, tabel, geom_type, tekst, størrelse, index, hoej_geom, sted, number_of_points, NumberOfInteriorPolygon, TotalNumberOfInteriorPolygons] # add the point,id_lokalid and the index of the point in the linestring to a list
                new_points.append(new)
            
            
# To Pandas DataFrame
data_points_interior = pd.DataFrame(new_points)

# Renaming columns
data_points_interior.rename(columns = {0 : 'geometry', 1 : 'id_lokalid', 2 : 'id_point_list', 3 : 'tabel', 4 : 'geom_type', 5 : 'tekst', 6 : 'Størrelse', 7 : 'index', 8 : 'hoej_geom', 9 : 'sted', 10 : 'Number_of_points', 11 : 'NumberOfInteriorPolygon', 12 :'TotalNumberOfInteriorPolygons'}, inplace = True)

# To GeoDataFrame
data_points_interior = gpd.GeoDataFrame(
    data_points_interior, geometry=data_points_interior['geometry'])

data_points_interior['Exterior'] = 'No'

#checking how many points we have
print(len(data_points_interior))

35982


## Interior, Points To LineSegments and Check if Object crosses Hoejspaending

In [78]:
# Get x, y, z coords
data_points_interior['x_coord'] = data_points_interior['geometry'].x
data_points_interior['y_coord'] = data_points_interior['geometry'].y
data_points_interior['z_coord'] = data_points_interior['geometry'].z

In [79]:
# Group by index and numberOfinteriorPolygon since some objects have multiple entities and multiple interiorPolygons

data_points_interior['index'] = data_points_interior['index'].astype(str)

data_points_interior['NumberOfInteriorPolygon'] =  data_points_interior['NumberOfInteriorPolygon'].astype(str)

data_points_interior['group'] = data_points_interior['index'] + data_points_interior['NumberOfInteriorPolygon']

# Groupby index and Interior Polygon
data_points_interior_group = data_points_interior.groupby('group').agg(lambda x: list(x))

In [80]:
change_points2 = []


# Points To LineSegments
for i, row in data_points_interior_group.iterrows():
    
    counter = 1

    # Sted to GeoSeries !
    sted_geom = geopandas.GeoSeries(row['sted'][0])
    sted_geom = sted_geom.set_crs("EPSG:25832")
    sted_geom = sted_geom.buffer(5)
    
    
        
    for point, id_point_list, Z_Størrelse in zip(row['geometry'], row['id_point_list'], row['Størrelse']):
        
        
        # We have to break the loop when we run out of points!
        if counter > row['id_point_list'][-1]:
            break
            
        else:
            
            # The counter is used to pick the point in front of the current point, So that we can make a LineString of the current point and the next point in the list 
            Line = LineString([point, row['geometry'][counter]])
               
            # The line we just created to GeoSeries
            Line_GeoSeries = geopandas.GeoSeries(Line)
            Line_GeoSeries = Line_GeoSeries.set_crs("EPSG:25832")
            
            # Add one to counter
            counter = counter + 1
            
            # check if our Linesegment crosses the buffed point sted_geom
            for _bool in Line_GeoSeries.crosses(sted_geom, align=True):
                if (_bool == True):
                    new2 = [i, row['id_lokalid'][0], id_point_list, row['id_point_list'][counter - 1], Z_Størrelse, row['NumberOfInteriorPolygon'][0]]
                    change_points2.append(new2)

## Interior, Bool_crossing Dataframe

In [81]:
# To Pandas DataFrame
Bool_crossing_interior = pd.DataFrame(change_points2)

# Renaming columns
Bool_crossing_interior.rename(columns = {0 : 'Index', 1 : 'objekt_id_lokalid', 2 : 'id_point_list_first_point', 3 : 'id_point_list_secound_point', 4 : 'Z_Størrelse', 5 : 'NumberOfInteriorPolygon'}, inplace = True)

In [82]:
# Handeling Z_størrelse where its written in cm, if the diffrence is less than 1m its written in centimeters so we are just converting it to centimeters. Bigger than five is because then it must be written in CM!
Bool_crossing_interior['Z'] = Bool_crossing_interior['Z_Størrelse'].map(lambda x: x.lstrip('Z difference: ').rstrip('cm'))

Bool_crossing_interior['Z'] = Bool_crossing_interior['Z'].str.replace(',', '.')

Bool_crossing_interior['Z'] = Bool_crossing_interior['Z'].astype(float)

Bool_crossing_interior['Z'] =  np.where(Bool_crossing_interior['Z'] > 5, Bool_crossing_interior['Z'] / 100, Bool_crossing_interior['Z'])

In [83]:
#with pd.option_context("display.max_rows", None):
#    display(Bool_crossing_interior)

## Interior, Merge Bool_crossing with data_points_interior_group

In [84]:
# Groupby index
Bool_crossing_interior_group = Bool_crossing_interior.groupby('Index').agg(lambda x: list(x))

In [85]:
done_interior = Bool_crossing_interior_group.join(data_points_interior_group, lsuffix='_left', rsuffix='_right')

print(len(done_interior))

29


In [86]:
# Subtract the minimum value in the list column by 1 and add it to the list
done_interior['id_point_list_first_point'] = done_interior['id_point_list_first_point'].apply(lambda x: x + [min(x) - 1])

# plus the max value in the list column by 1 and add it to the list
done_interior['id_point_list_secound_point'] = done_interior['id_point_list_secound_point'].apply(lambda x: x + [max(x) + 1])

## Interior, Lodrette Vektore

In [87]:
def zero_with_max(lst, max_val):
    if 0 in lst:
        lst.append(max_val)
    return lst

def max_with_zero(lst, max_val):
    if max_val in lst:
        lst.append(0)
    return lst

In [88]:
done_interior['Max_Value'] = done_interior['id_point_list'].apply(lambda x: max(x))

done_interior['id_point_list_first_point'] = done_interior.apply(lambda row: zero_with_max(row['id_point_list_first_point'], row['Max_Value']), axis=1)

done_interior['id_point_list_secound_point'] = done_interior.apply(lambda row: max_with_zero(row['id_point_list_secound_point'], row['Max_Value']), axis=1)

## Interior, Edit the Z Coord

In [89]:
change_z_interior = []

for i, row in done_interior.iterrows():
    for id_point, x, y, z, geom, num in zip(row['id_point_list'], row['x_coord'], row['y_coord'], row['z_coord'], row['geometry'], row['NumberOfInteriorPolygon_right']):
        if (id_point in row['id_point_list_first_point'] or id_point in row['id_point_list_secound_point']):
            new_z = max(round(z + row['Z'][0] - 10, 2), 2)
            coords = (x, y, new_z)
            point = Point(coords)
            changed = 'Yes'
            new_changed_z = [i, id_point, point, changed, row['tabel'][0], row['geom_type'][0], row['id_lokalid'][0], new_z, num]
            change_z_interior.append(new_changed_z)
        else:
            changed = 'No'
            not_change_z = [i, id_point, geom, changed, row['tabel'][0], row['geom_type'][0], row['id_lokalid'][0], z, num]
            change_z_interior.append(not_change_z)
            
       
# To Pandas DataFrame
edit_z_interior = pd.DataFrame(change_z_interior)

# Renaming columns
edit_z_interior.rename(columns = {0 : 'group', 1 : 'id_point_list', 2 : 'geometry', 3 : 'Changed', 4 : 'tabel', 5 : 'geom_type', 6 : 'Id_Lokalid', 7 : 'Z', 8: 'NumberOfInteriorPolygon'}, inplace = True)            

# WKT
#edit_z_interior['geometry'] = edit_z_interior['geometry'].apply(wkt.loads)

# To GeoDataFrame
edit_z_interior = gpd.GeoDataFrame(
    edit_z_interior, geometry=edit_z_interior['geometry'])

In [90]:
# Groupby index
edit_z_interior_group = edit_z_interior.groupby('group').agg(lambda x: list(x))
edit_z_interior_group.reset_index(inplace=True)

## Interior, handle objects who have multiple entries

In [91]:
edit_z_interior_group['id_lokalid'] = edit_z_interior_group['Id_Lokalid'].map(lambda x: x[0])
edit_z_interior_group['NumberOfInteriorPolygon'] = edit_z_interior_group['NumberOfInteriorPolygon'].map(lambda x: x[0])

In [92]:
# Groupby 'id_lokalid'
grouped = edit_z_interior_group.groupby(['id_lokalid', 'NumberOfInteriorPolygon']).agg(lambda x: list(x))

In [93]:
print('Der Findes i alt', len(edit_z_interior_group), 'Punkter som krydser interior objekterne dog findes der kun', len(grouped), 'unikke interior objekter så nogle objekter bliver krydset flere steder, disser har multiple entries')

Der Findes i alt 29 Punkter som krydser interior objekterne dog findes der kun 18 unikke interior objekter så nogle objekter bliver krydset flere steder, disser har multiple entries


In [94]:
# The function is called in the for loop below !
def create_namedtuple(name, fields, values, fillvalue):
    TupleType = namedtuple(name, fields)
    data = zip_longest(fields, values, fillvalue=fillvalue)
    lists = [item[1] for item in TupleType(*data)]
              
    return lists

In [95]:
done_done = []

# IF some objects have more than 10 Entities update the 'fields' List with more fields ect.. list11, list12 !! 
fields = ['list1', 'list2', 'list3', 'list4', 'list5', 'list6', 'list7', 'list8', 'list9', 'list10']


# The Dataframe grouped have some collumns with neasted list's (list's inside a list)
# The problem is that our neasted list's have diffrent length's therfore we have to use the function 'create_namedtuple'
# To create List that have equal lenght.
for q, row in grouped.iterrows():
    
    #print('---------------', len(row['Index']), '--------------')
    #print('id_lokalid ;', q)
    #print(row['id_point_list'][0])
    
    geom_type = row['geom_type'][0]
    
    tabel = row['tabel'][0]
    
    #print(tabel)
    
    
    if (len(row['group']) > 1):
        # If the object have multiple entities
        
        #print(len(row['Index']))
        
        
        # fillvalue is used in our function 'create_namedtuple'
        n  = len(row['Changed'][0])
        
        
        fillvalue = ['X' for x in range(n)] 
        
        # create_namedtuple for row['Changed']
        Changed = create_namedtuple('MyTuple_Changed', fields, row['Changed'], fillvalue)
        
        # create_namedtuple for row['geometry']
        Points = create_namedtuple('MyTuple_Points', fields, row['geometry'], fillvalue)
        
        # create_namedtuple for row['id_point_list'] 
        id_point_list = create_namedtuple('MyTuple_id_point_list', fields, row['id_point_list'], fillvalue)
        
        
        
        # used to append the list done to the list done_done outside the for loop
        #counter = 2
        
        # List outside the Neasted for loop
        done = []
        

        # for elements in all the Named Tupels
        for a, b, c, d, e, f, g, h, i, j, a1, b1, c1, d1, e1, f1, g1, h1, i1, j1, a2, b2, c2, d2, e2, f2, g2, h2, i2, j2 in zip(*Changed, *Points, *id_point_list):
            
   
            # Changed to list
            Changed_entries = [a, b, c, d, e, f, g, h, i, j]
            
            # Points to list
            Points_entries = [a1, b1, c1, d1, e1, f1, g1, h1, i1, j1]
            
            # Id_point_list
            id_point_list_entries = [a2, b2, c2, d2, e2, f2, g2, h2, i2, j2]
            
            #print(Changed_entries)
            #print(id_point_list_entries)
            
            
            
            # if 'Yes' in Changed_entries (there are changed points in the list 'Points_entries')
            if ('Yes' in Changed_entries):
                
                
                #print(Changed_entries)
                #print(id_point_list_entries)
                
                # Getting the index of the changed points, since these equals the index of 'Yes' in the list 'index_of_yes'
                index_of_yes = [i for i, v in enumerate(Changed_entries) if v == 'Yes']
                #print(f'Index of Yes: {index_of_yes}')
                
                
                # Getting only the changed Points in a seprate list 'result'
                ids = index_of_yes
                values = Points_entries
                
                # Logic ---> value for index, value in enumarate(list1) if index in list2
                result = [val for i, val in enumerate(values) if i in ids]
                
                #print(result)
                
                # Getting the Point with the smallest Z value from the list of points 'result'
                min_z_point = min(result, key=lambda p: p.z)
                
                #print('Min z point :', min_z_point)
                
                # Append the id_lokalid, id_point_list and minimum z coord point to the list 'done'
                row_done = [q, id_point_list_entries[0], min_z_point, geom_type[0], tabel[0]]
                done.append(row_done)
                
                #print(done)
                #print(len(done), 'lenght of done')
                
                # Appending the list done to the list done_done if counter equal len
                if (len(done) == n):
                     # To Pandas DataFrame
                    done_pd = pd.DataFrame(done)
                    
                    # Renaming columns
                    done_pd.rename(columns = {0 : 'Id_Lokalid', 1 : 'id_point_list', 2 : 'geometry', 3: 'geom_type', 4 : 'tabel'}, inplace = True)
                    
                    #print(done_pd)
                    
                  
                    #print(len(done), n, 'append outside loop, yes in changed')
                    done_done.append(done_pd)
 
            else:
                # Append the id_lokalid, id_point_list and point to the list 'done'
                row_done = [q, id_point_list_entries[0], Points_entries[0], geom_type[0], tabel[0]]
                
                done.append(row_done)

                if (len(done) == n):
                    
                    # To Pandas DataFrame
                    done_pd = pd.DataFrame(done)

                    # Renaming columns
                    done_pd.rename(columns = {0 : 'Id_Lokalid', 1 : 'id_point_list', 2 : 'geometry', 3: 'geom_type', 4 : 'tabel'}, inplace = True) 
                    
                    #print(done_pd)
                    
                    
                    #print(len(done), n, 'append outside loop, not yes in changed')
                    done_done.append(done_pd)
                    
    else:
        
        done = []
        
        #print('ONLY ONE ENTITY')
        
        for a, b in zip(row['id_point_list'][0], row['geometry'][0]):

            row_done = [q, a, b, geom_type[0], tabel[0]]

            done.append(row_done)

            if(len(row['Changed'][0]) == len(done)):
                
                
                
                # To Pandas DataFrame
                done_pd = pd.DataFrame(done)

                # Renaming columns
                done_pd.rename(columns = {0 : 'Id_Lokalid', 1 : 'id_point_list', 2 : 'geometry', 3 : 'geom_type', 4 : 'tabel'}, inplace = True)

                #print(done_pd)

                #print(len(done), len(row['Changed'][0]), 'append outside loop, only one entity')
                #print(done_pd)

                done_done.append(done_pd)

In [96]:
# To Pandas DataFrame
multiple_entries_interior = pd.concat(done_done)


multiple_entries_interior['NumberOfInteriorPolygon'] = multiple_entries_interior['Id_Lokalid'].apply(lambda x: x[1])

multiple_entries_interior['Id_Lokalid'] = multiple_entries_interior['Id_Lokalid'].apply(lambda x: x[0])


# group by id_lokalid
multiple_entries_group_interior = multiple_entries_interior.groupby(['Id_Lokalid', 'NumberOfInteriorPolygon']).agg(lambda x: list(x))

assert len(multiple_entries_group_interior) == len(grouped), 'These dataframes should have same length'

In [97]:
# Checking if id_point_list is sorted right!

for i, row in  multiple_entries_group_interior.iterrows():
    counter = 0
    for element, nxt in zip(row['id_point_list'], row['id_point_list'][1:]):
        while counter == 0:
            if (counter != element):
                print(counter, element)
            break
        counter = counter + 1
        if (counter != nxt):
            print(counter, nxt)

## interior, Get all the interior polygons 
Join on Id_lokalid and numberofinteriorPolygon

In [98]:
interior_result1 = []

for i, row in result_1.iterrows():
    if(len(row['coord_list_objects_interior']) > 0):
        for index, element in enumerate(row['coord_list_objects_interior']):
            interior_polygons = [row['objekt_id_lokalid'] ,len(row['coord_list_objects_interior']), index + 1, element, row['geom_type_objekt'], row['objekt']]
            interior_result1.append(interior_polygons)
            
# To Pandas DataFrame
All_interior = pd.DataFrame(interior_result1)

# Renaming columns
All_interior.rename(columns = {0 : 'Id_Lokalid', 1 : 'TotalNumberOfInterior', 2 : 'NumberOfInteriorPolygon', 3 : 'geometry', 4 : 'geom_type', 5 : 'table'}, inplace = True)


In [99]:
# only get unique interior polygons out based on 'Id_Lokalid', 'NumberOfInteriorPolygon'
All_interior = All_interior.drop_duplicates(subset=['Id_Lokalid', 'NumberOfInteriorPolygon'])

In [100]:
multiple_entries_group_interior = multiple_entries_group_interior.reset_index()

multiple_entries_group_interior['Id_Lokalid'] = multiple_entries_group_interior['Id_Lokalid'].astype(int)

multiple_entries_group_interior['NumberOfInteriorPolygon'] = multiple_entries_group_interior['NumberOfInteriorPolygon'].astype(int)

All_interior['Id_Lokalid'] = All_interior['Id_Lokalid'].astype(int)

All_interior['NumberOfInteriorPolygon'] = All_interior['NumberOfInteriorPolygon'].astype(int)

In [101]:
merged_interior = pd.merge(All_interior, multiple_entries_group_interior, on=['Id_Lokalid', 'NumberOfInteriorPolygon'], how='left')

In [102]:
merged_interior['geometry_y'] = merged_interior['geometry_y'].fillna('xy')

In [103]:
pointlist_interior = []

id_pointlist_interior = []

for i, row in merged_interior.iterrows():
    
    point_list = []
    id_point_list = []
    
    if (row['geometry_y'] == 'xy'):
        
      
        
        for id_point, coord in enumerate(row['geometry_x']):
        
            point = Point(coord)
            
            point_list.append(point)
            
            id_point_list.append(id_point)
            
        if (len(id_point_list) == len(row['geometry_x'])):
            pointlist_interior.append(point_list)
            id_pointlist_interior.append(id_point_list)
    else:
        pointlist_interior.append(row['geometry_y'])
        
        id_point_list = []
        
        for id_point, coord in enumerate(row['geometry_y']):
            
            id_point_list.append(id_point)
            
            if (len(id_point_list) == len(row['geometry_y'])):
           
                id_pointlist_interior.append(id_point_list)
            
    
    

In [104]:
merged_interior['new_geometry'] = pointlist_interior

merged_interior['new_geometry_id_pointlist'] = id_pointlist_interior

In [105]:
merged_interior = merged_interior[['Id_Lokalid', 'TotalNumberOfInterior', 'NumberOfInteriorPolygon', 'table', 'new_geometry', 'new_geometry_id_pointlist']]

In [106]:
from shapely import geometry

def creat_geom(x): 
    '''
    This function Creats polygons based the a list of points
    '''
        
    Poly = geometry.Polygon([[p.x, p.y, p.z] for p in x]) 
        
    return Poly



merged_interior['Geom_interior'] =  merged_interior.apply(lambda x: creat_geom(x['new_geometry']),axis=1)

In [107]:
merged_interior = merged_interior.groupby('Id_Lokalid').agg(lambda x: list(x))

In [108]:
merged_interior.rename(columns={"new_geometry": "interior_geometry", "new_geometry_id_pointlist": "interior_id_pointlist"}, inplace=True)

# Join  Interior and Exterior Polygons

In [109]:
multiple_entries_group['Id_Lokalid'] = multiple_entries_group.index

multiple_entries_group['Id_Lokalid'] = multiple_entries_group['Id_Lokalid'].astype(str)

multiple_entries_group = multiple_entries_group.reset_index(drop=True)


In [110]:
merged_interior['Id_Lokalid'] = merged_interior.index

merged_interior['Id_Lokalid'] = merged_interior['Id_Lokalid'].astype(str)

merged_interior = merged_interior.reset_index(drop=True)

In [111]:
Everything = pd.merge(multiple_entries_group, merged_interior, on=['Id_Lokalid'], how='left')

# From Pointlist To LineStrings and Polygons

In [112]:
Everything['geom_type'] = Everything['geom_type'].apply(lambda x: x[0])

Everything['tabel'] = Everything['tabel'].apply(lambda x: x[0])

In [113]:
from shapely import geometry

def creat_geom(x, y): 
    '''
    This function Creats linestrings and polygons based on the geom_type collumn,
    x = geom_type and y = Point list
    '''
    
    if x == 'LineString':
        
        Line = geometry.LineString([[p.x, p.y, p.z] for p in y])
        
        return Line
    
    elif x == 'Polygon':
        
        Poly = geometry.Polygon([[p.x, p.y, p.z] for p in y]) 
        
        return Poly



Everything['Geom_exterior'] =  Everything.apply(lambda x: creat_geom(x['geom_type'],x['geometry']),axis=1)

In [114]:
#Everything_test = Everything.loc[Everything['Id_Lokalid'] == '1219452582']

In [115]:
Everything['Geom_interior'].fillna('x', inplace=True)

# Create Geometry with Both Interior and Exterior Polygons

In [116]:
#Everything_test = Everything.loc[Everything['Id_Lokalid'] == '1219452582']

In [117]:
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from shapely.errors import GEOSException

geometry_done = []

for i, row in Everything.iterrows():
    
    #print(row['NumberOfInteriorPolygon'])
    
    Geom_exterior = row['Geom_exterior']
    
    Geom_interior = row['Geom_interior']
    

    if row['Geom_interior'] != 'x':
    
        # CREATE THE NEW POLYGON WITH INNER HOLES
        newPolygon = Polygon(Geom_exterior.exterior.coords, [inner.exterior.coords for inner in Geom_interior])
                   
        geometry_done.append(newPolygon)
            
    else:
        
        geometry_done.append(Geom_exterior)
        
                        
       
    

In [118]:
Everything['Geometry_done'] = geometry_done

# Output

In [122]:
done = Everything[['Id_Lokalid', 'tabel', 'Geometry_done', 'geom_type']]

# To GeoDataFrame
done = gpd.GeoDataFrame(
done, geometry=done['Geometry_done'])
            
done.crs = "EPSG:25832"
    
done.drop(['Geometry_done'], axis=1, inplace=True) # droping geometri so we dont have both geometry and geometri column

fp_save =  r'C:/Users/b177999/Desktop/Hoejspaending_over_under/'

In [123]:
# Skov 
#done_skov = done.loc[done.tabel == 'SKOV'].copy()

#done_skov.to_file(fp_save + "Skov.shp", driver='ESRI Shapefile')

In [124]:
# Alle LineStings

#LineString_done = done.loc[done.geom_type == 'LineString'].copy()

#LineString_done.to_file(fp_save + "LineString.shp", driver='ESRI Shapefile')

In [ ]:
#LineString_done['tabel'].unique()

In [125]:
# Alle Polygoner

#Polygon_done = done.loc[done.geom_type == 'Polygon'].copy()

#Polygon_done.to_file(fp_save + "Polygon.shp", driver='ESRI Shapefile')

In [126]:
#Polygon_done['tabel'].unique()